# Tensorboard Visualization

In [1]:
import tensorflow as tf

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(\
        "/media/zheng/study/my_work/project/data/mnist", \
        one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /media/zheng/study/my_work/project/data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /media/zheng/study/my_work/project/data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /media/zheng/study/my_work/project/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /media/zheng/study/my_work/project/data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_epoch = 1
logs_path = "/media/zheng/study/my_work/project/logs/mnist"

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

In [3]:
# Create  model
def multilayer_perception(x, weights, biases):
    # Hidden layer with ReLU activation
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Create a summary to visualize the first layer ReLU activation
    tf.summary.histogram("relu1", layer_1)
    # Hidden layer with ReLU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Create another summary to visualize the second layer ReLU activation
    tf.summary.histogram("relu2", layer_2)
    # Output layer
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return out_layer

# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name = 'W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name = 'W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name = 'W3')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

In [4]:
# Encapsulating all ops into scopes, making Tensorboard's Graph
# Visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = multilayer_perception(x, weights, biases)
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Op to calculate every variable gradient
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    # Op to update all variables according to their gradient
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads)
    
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [5]:
# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Create summaries to visualize weights
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)

# Summarize all gradients
for grad, var in grads:
    print(grad, var)
    tf.summary.histogram(var.name + '/gradient', grad)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

INFO:tensorflow:Summary name W1:0 is illegal; using W1_0 instead.
INFO:tensorflow:Summary name W2:0 is illegal; using W2_0 instead.
INFO:tensorflow:Summary name W3:0 is illegal; using W3_0 instead.
INFO:tensorflow:Summary name b1:0 is illegal; using b1_0 instead.
INFO:tensorflow:Summary name b2:0 is illegal; using b2_0 instead.
INFO:tensorflow:Summary name b3:0 is illegal; using b3_0 instead.
Tensor("SGD/gradients/Model/MatMul_grad/MatMul_1:0", shape=(784, 256), dtype=float32) <tf.Variable 'W1:0' shape=(784, 256) dtype=float32_ref>
INFO:tensorflow:Summary name W1:0/gradient is illegal; using W1_0/gradient instead.
Tensor("SGD/gradients/Model/MatMul_1_grad/MatMul_1:0", shape=(256, 256), dtype=float32) <tf.Variable 'W2:0' shape=(256, 256) dtype=float32_ref>
INFO:tensorflow:Summary name W2:0/gradient is illegal; using W2_0/gradient instead.
Tensor("SGD/gradients/Model/MatMul_2_grad/MatMul_1:0", shape=(256, 10), dtype=float32) <tf.Variable 'W3:0' shape=(256, 10) dtype=float32_ref>
INFO:ten

In [6]:
# Start Training
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path,
                                           graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([apply_grads, loss, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Epoch: 0001 cost= 55.096309079
Epoch: 0002 cost= 13.164456361
Epoch: 0003 cost= 8.152336087
Epoch: 0004 cost= 5.781471548
Epoch: 0005 cost= 4.398238262
Epoch: 0006 cost= 3.455879622
Epoch: 0007 cost= 2.765935290
Epoch: 0008 cost= 2.252990000
Epoch: 0009 cost= 1.867038618
Epoch: 0010 cost= 1.598959076
Epoch: 0011 cost= 1.344374338
Epoch: 0012 cost= 1.122601289
Epoch: 0013 cost= 0.997937928
Epoch: 0014 cost= 0.841046343
Epoch: 0015 cost= 0.726039742
Epoch: 0016 cost= 0.623193720
Epoch: 0017 cost= 0.547792133
Epoch: 0018 cost= 0.482531930
Epoch: 0019 cost= 0.415341744
Epoch: 0020 cost= 0.369240061
Epoch: 0021 cost= 0.309209254
Epoch: 0022 cost= 0.282216173
Epoch: 0023 cost= 0.244210703
Epoch: 0024 cost= 0.211156987
Epoch: 0025 cost= 0.187476667
Optimization Finished!
Accuracy: 0.9289
Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs 
Then open http://0.0.0.0:6006/ into your web browser
